Main code

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

number_of_days = '8d'
check_last_x = -100

def save_messages_to_json(messages, filename): # Dump messages to the json file
    with open(filename, 'a', encoding='utf-8') as f:
        for message in messages:
            json.dump(message, f, ensure_ascii=False)
            f.write('\n')

def log_into_account(driver, username, password):
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, 'login'))).send_keys(username)
    driver.find_element(By.NAME, 'password').send_keys(password)
    driver.find_element(By.XPATH, '//button[@data-testid="log-in-submit"]').click()
    WebDriverWait(driver, 10).until(EC.url_changes(driver.current_url))

def scroll_and_scrape(driver, filename, saved_message_ids):
    messages = driver.find_elements(By.CSS_SELECTOR, 'div.StreamMessage_container__omTCg')[check_last_x:] # Protection to speed things up (You can try adjusting the number)
    new_messages = []
    found_desired_timestamp = False

    for message in messages:
        try:
            message_id = message.get_attribute('data-testid').replace('message-', '') # Extract unique message ids
            if message_id in saved_message_ids: #look for duplicates and skip them
                continue
            saved_message_ids.add(message_id)
            saved_message_ids = saved_message_ids[check_last_x:]

            # Extracting important information
            message_text = message.find_element(By.CSS_SELECTOR, 'div.RichTextMessage_body__4qUeP').text
            timestamp = message.find_element(By.CSS_SELECTOR, 'time.StreamMessage_timestamp__VVDmF').text
            likes_elem = message.find_elements(By.CSS_SELECTOR, 'span.StreamMessageLabelCount_labelCount__dWyPL')
            likes = likes_elem[0].text if likes_elem else '0'

            if timestamp == number_of_days:
                found_desired_timestamp = True

            message_obj = {
                'id': message_id,
                'text': message_text,
                'timestamp': timestamp,
                'likes': likes
            }
            new_messages.append(message_obj)

        except Exception:
            continue

    if new_messages:
        save_messages_to_json(new_messages, filename) # Save new messages

    return found_desired_timestamp

def remove_old_messages(driver): # Saves RAM and makes scraping much more efficient and faster
    # JavaScript to remove messages that are out of the viewport
    script = """
    var messages = document.querySelectorAll('div.StreamMessage_container__omTCg');
    var viewportHeight = window.innerHeight;
    messages.forEach(function(message) {
        var rect = message.getBoundingClientRect();
        if (rect.bottom < 0) {
            message.parentNode.removeChild(message);
        }
    });
    """
    driver.execute_script(script)

def main():
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get('https://stocktwits.com/signin?next=/login')

    with open('usrpass.txt', 'r') as file:
        username, password = [line.strip() for line in file.readlines()]

    log_into_account(driver, username, password)
    driver.get('https://stocktwits.com/stream/trending')

    json_filename = 'messages.json'
    saved_message_ids = set()

    try:
        with open(json_filename, 'r', encoding='utf-8') as f:
            for line in f:
                saved_message_ids.add(json.loads(line)['id'])
    except FileNotFoundError:
        pass

    scroll_count = 0
    while True:
        if scroll_and_scrape(driver, json_filename, saved_message_ids):
            print("Reached desired timestamp.")
            break
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        scroll_count += 1

        # Remove old messages every few scrolls to reduce memory usage
        if scroll_count % 5 == 0:
            remove_old_messages(driver)

        # Wait for new messages to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.StreamMessage_container__omTCg'))
        )

    print("Scraping completed.")
    driver.quit()

if __name__ == "__main__":
    main()